# Projeto 1 - Ciência dos Dados

Nome: Luana Mitsudo Coelho

Nome: Vinicius Laranjeira Cardoso

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [78]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

import functools
import operator
import re
import emoji

In [79]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Luana Mitsudo\Documents\INSPER\2.2 sem\Cdados\Projeto 1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [80]:
filename = 'nespresso.xlsx'

In [81]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Valor
0,@peter_eller10 nespresso,0
1,@oicronofobico eu tomo o nespresso decaf saben...,1
2,café fortaleza platinium – cápsulas compatible...,0
3,@_larisantiago dolce gusto é muito boa hahahah...,1
4,@barbaramarlene_ @mpcouto_ @olhinhos_dmel_1 ya...,0


In [82]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Valor
0,@philatticus nespresso?!,0
1,@fofinha_srta nespresso,0
2,@sneaknmeeks @ghaspermusic nespresso,0
3,@liliusko nespresso,0
4,100 cápsulas de café 100% arábica para nespres...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [83]:
#Limpeza do Data Frame
import re 
def cleanup(text):
    punctuation = '[!-.:?;]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [84]:
#Limpeza apenas do Treinamento
total_treinamento = " ".join(train.Treinamento).lower().split()

In [85]:
#Remoção dos emoji
em_split_whitespace = [substr.split() for substr in total_treinamento]
palavras_treinamento = functools.reduce(operator.concat, em_split_whitespace)

In [86]:
#Probabilidade de um tweet ser relevante ou irrelevante
prob = train.Valor.value_counts(True, sort=False)
P_I = prob[0]
P_R = prob[1]
print(P_I)
print(P_R)

0.5566666666666666
0.44333333333333336


In [87]:
# Convertendo treinamento em pd.Series
serie_treinamento = pd.Series(palavras_treinamento)
serie_treinamento

0       @peter_eller10
1            nespresso
2       @oicronofobico
3                   eu
4                 tomo
             ...      
6197          incrivel
6198               etc
6199               etc
6200                 e
6201              tals
Length: 6202, dtype: object

In [88]:
# Tabela absoluta do treinamento
tabela_treinamento_absoluta = serie_treinamento.value_counts()
tabela_treinamento_absoluta.head()

de           224
nespresso    197
a            151
e            140
que          129
dtype: int64

In [89]:
#Separação dos tweets em relevantes
filtro1 = train['Valor'] == 1
dados_R = train.loc[filtro1,:]

In [90]:
#Separação dos tweets em irrelevantes
filtro2 = train['Valor'] == 0
dados_I = train.loc[filtro2,:]

In [91]:
# Organizando a base de dados separada em relevantes e irrelevantes
palavras_R = " ".join(dados_R.Treinamento).lower().split()
palavras_I = " ".join(dados_I.Treinamento).lower().split()

In [92]:
# Tabela de frequências para palavras relevantes
tabela_R = pd.Series(palavras_R).value_counts()
tabela_R.head()

de           121
a            105
nespresso     98
que           94
e             89
dtype: int64

In [93]:
# Tabela de frequências para palavras irrelevantes
tabela_I = pd.Series(palavras_I).value_counts()
tabela_I.head()

de           103
nespresso     99
o             55
e             51
a             46
dtype: int64

In [94]:
#total de palavras relevantes
total_R = len(set(palavras_R))

In [95]:
#total de palavras irrelevantes
total_I = len(set(palavras_I))

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [96]:
#Remoção dos emoji
lista_tweets_limpos = []
for index, linha in test.iterrows():
    tweet = cleanup(linha.Teste.lower()).split()
    em_split_whitespace = [substr.split() for substr in tweet]
    palavras_teste = functools.reduce(operator.concat, em_split_whitespace)
    lista_tweets_limpos.append(palavras_teste)

In [99]:
#Suavização de Laplace
i = 0
n = 0

while i < len(lista_tweets_limpos):
    prob_tweet_R = 1
    prob_tweet_I = 1
    
    for n in (lista_tweets_limpos[i]):
        if n not in tabela_R:
            prob_tweet_R *= (0+1)/(total_R+(total_R+total_I))
        else:
            prob_tweet_R *= (tabela_R[n]+1)/(total_R+(total_R+total_I))
        if n not in tabela_I:
            prob_tweet_I *= (0+1)/(total_I+(total_R+total_I))
        else:
            prob_tweet_I *= (tabela_I[n]+1)/(total_I+(total_R+total_I))
    if (prob_tweet_R*P_R) > (prob_tweet_I*P_I):
        test.loc[i, 'Classificador'] = 1
    else:
        test.loc[i, 'Classificador'] = 0
        
    i += 1

In [100]:
#Tabela comparativa do treinamento com a teste
tabela = pd.crosstab(test['Classificador'], test['Valor'])

In [101]:
tabela

Valor,0,1
Classificador,,
0.0,56,3
1.0,96,49


In [105]:
verdadeiro_positivo = (tabela[1][1]/(tabela[0][1]+tabela[1][1]))
falso_positivo = (tabela[0][1]/(tabela[0][1]+tabela[1][1]))
verdadeiro_negativo = (tabela[0][0]/(tabela[0][0]+tabela[1][0]))
falso_negativo = (tabela[1][0]/(tabela[1][0]+tabela[0][0]))

print(f'Werdadeiros positivos = {verdadeiro_positivo}')
print(f'Falsos positivos = {falso_positivo}')
print(f'Verdadeiros negativos = {verdadeiro_negativo}')  
print(f'Falsos negativos = {falso_negativo}')

Werdadeiros positivos = 0.33793103448275863
Falsos positivos = 0.6620689655172414
Verdadeiros negativos = 0.9491525423728814
Falsos negativos = 0.05084745762711865


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

https://www.w3schools.com/python/ref_string_join.asp

https://stackoverflow.com/questions/49921720/how-to-split-emoji-from-each-other-python